# Sample app

This notebook is meant as a sample to show how a bearer token (JWT) can be created in Python using a Service Principal its Client Id and Client Secret.  
The token can then be used to interact with the backend service, `Sample.Manufacturing.Dotnet.Api`.

## Prerequisites

As mentioned in the [Readme](/README.md), role-based security is used in the backend service. Therefore, the Service Principal needs to have the appropriate roles assigned to its identity. Assigning these roles is described in the [Microsoft Learn pages](https://learn.microsoft.com/en-us/graph/api/serviceprincipal-post-approleassignments?view=graph-rest-1.0&tabs=http) on this subject. A sample script can also be found over here in the [deployment scripts of the Secure-Api-repository](https://github.com/Jandev/secure-apis/blob/aafa11d5b038e8833dba061ea3054dce31258731/deployment/AAD/create-apps.ps1#L160).

## Import the dependencies

In [ ]:
pip install azure-identity

In [ ]:
from azure.identity import ClientSecretCredential
import requests
import urllib3

### Add the variables used in this sample

In [ ]:
# Used for retrieving the token.
tenant_id="<The tenant id where the applications are deployed to>"
client_id="<The Client Id of the Partner App Registration>"
client_secret="<The Client Secret created for the Partner App Registration>"
serviceAppUri = "<The App ID URI of the backend solution>"

In [ ]:
# Used to invoke the backend API.
backendApiUrl = "<The endpoint of backend that needs to be invoked.>"

## Do the work

Create a token for the Service Principal, and make an HTTP request to the backend service.

In [ ]:
serviceAppUri = serviceAppUri
credential = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret,
)

bearerToken = credential.get_token(f"{serviceAppUri}.default")


### Get the response from the backend API

In [ ]:
headers = {"Authorization": f"Bearer {bearerToken.token}"}

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

try:
    response = requests.get(backendApiUrl, headers=headers, verify=False)
    response.raise_for_status()
    print(response.json())
except requests.exceptions.HTTPError as err:
    print(f"HTTP Error: {err}")
except requests.exceptions.RequestException as err:
    print(f"Request Exception: {err}")
